In [15]:
import pandas as pd
import re
import torch
import numpy as np
import torch
import transformers
from ast import literal_eval
import json
import pandas as pd
import random


pd.set_option('display.max_columns', None)

In [24]:
df = pd.read_csv("combined_sorted_data.csv")

In [25]:
df1 = pd.read_csv("bad_paragraph.csv")

In [26]:
df

,id,paragraph,question,choices,answer,question_plus,cleaned_category
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,NaN,사회-한국사
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,사회-동아시아사
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,사회-동아시아사
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,사회-한국지리
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,사회-동아시아사
...,...,...,...,...,...,...,...
2016,generation-for-nlp-2893,"“헐값에 팔리는 냉동 오렌지주스만 잔뜩 사가고, 쿠폰을 오려 모았으며, 구멍 난 스...",잭 맥도널드가 남긴 유산의 총액은 얼마인가?,"['1억8760만달러', '5000만달러', '2억달러', '1억달러', '3억달러']",1,NaN,사회-경제
2017,generation-for-nlp-2894,"넷기어코리아(지사장 김진겸, 이하 넷기어)가 뮤럴 캔버스를 가지고 넷기어 SNS에서...",넷기어가 모집하는 '뮤럴 공식 도슨트'의 주요 역할은 무엇인가?,"['전시물 설명 및 안내', '고객 서비스 제공', '제품 판매 촉진', '온라인 ...",4,NaN,사회-문화
2018,generation-for-nlp-2895,서울 성동구 옥수동과 금호동은 맞붙어 있는 동네지만 아파트값은 같은 면적에서 1억원...,옥수동과 금호동의 아파트값 차이가 발생한 주된 이유는 무엇인가?,"['옥수동의 재개발이 먼저 이루어졌다', '금호동의 인구가 더 많다', '옥수동이 ...",1,NaN,사회-경제
2019,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?,"['셀트리온', '삼성전자', 'LG화학', '한화그룹', '현대자동차']",1,NaN,사회-경제


In [27]:
df1

,id,paragraph,question,choices,answer,question_plus
0,generation-for-nlp-662,대체출산이 달성된 지 오래된 후에도 인구가 오랫동안 계속해서 증가하는 경향은...,대체출산이 달성된 지 오래된 후에도 인구가 오랫동안 계속해서 증가하는 경향은?,"['영의 인구 증가', '고성장 속도', '항상성 고원', '인구학적 모멘텀']",4,NaN
1,generation-for-nlp-663,거리가 멀어짐에 따라 이동이 감소하는 경향은...,거리가 멀어짐에 따라 이동이 감소하는 경향은?,"['유출 요인', '유인 요인', '거리 감쇠', '이수 선택성']",3,NaN
2,generation-for-nlp-664,다국적 기업은 운영을 분산하여...,다국적 기업의 운영 방식에 대한 설명으로 옳은 것은?,"['생산비용이 가장 높은 제품을 제조한다', '경제적인 곳에서 회계 및 연구 서비스...",3,NaN
3,generation-for-nlp-667,인도 바라나시(베나레스)와 사우디아라비아의 메카가 비슷한 도시인 이유는...,인도 바라나시(베나레스)와 사우디아라비아의 메카가 비슷한 도시인 이유는?,"['구 영국 식민지였던 국가의 수도', '수많은 순례자의 목적지', '세계 경제의 ...",2,NaN
4,generation-for-nlp-675,"선거인단 제도의 구조를 감안할 때, 대선 후보들은 주로 어떤 경향을 갖습니까?","선거인단 제도의 구조를 감안할 때, 대선 후보들은 주로 어떤 경향을 갖습니까?","['격전지를 피하고 “안전한” 주에 선거운동을 집중', '인구가 많은 주에서 더 많...",2,NaN
5,generation-for-nlp-677,수정헌법 제1조는 언론과 종교의 자유를 보호하는 것으로 가장 잘 알려져 있는데.,수정헌법 제1조가 보호하는 또 다른 권리는 무엇입니까?,"['모든 형사 사건에서의 적법절차', '정부가 정당하게 보상하지 않는 한 개인 재산...",4,NaN
6,generation-for-nlp-683,당좌 예금 계좌에 1000달러가 입금되고 초과 준비금이 700달러 증가하면 준비율은...,당좌 예금 계좌에 1000달러가 입금되고 초과 준비금이 700달러 증가하면 준비율은...,"['70%.', '30%.', '40%.', '90%.']",2,NaN
7,generation-for-nlp-688,총 공급 곡선이 왼쪽으로 이동하면 균형 물가 수준과 균형 산출량은 어떻게 되겠습니까...,총 공급 곡선이 왼쪽으로 이동하면 균형 물가 수준과 균형 산출량은 어떻게 되겠습니까?,"['균형 물가 수준은 증가하는 반면 균형 산출량은 감소한다.', '균형 물가 수준은...",1,NaN
8,generation-for-nlp-691,단기 총 공급 곡선이 수평인 경우 그 이유는 다음과 같습니다.,단기 총 공급 곡선이 수평인 경우 그 이유는?,['임금과 물가를 올리지 않고도 생산량을 늘릴 수 있도록 유휴 자원이 많이 존재한다...,1,NaN
9,generation-for-nlp-693,사람들이 가까운 시일 내에 특정 제품의 가격이 인상될 것으로 예상하는 경우,사람들이 가까운 시일 내에 특정 제품의 가격이 인상될 것으로 예상하는 경우에 대한 ...,"['이는 현재 제품 수요에는 영향을 미치지 않는다.', '이는 제품에 대한 수요를 ...",4,NaN


In [28]:
import pandas as pd

# df와 df1 데이터프레임이 이미 생성되어 있다고 가정합니다.

# df에서 df1과 paragraph가 동일한 데이터 찾기
mask = df['paragraph'].isin(df1['paragraph'])

# 동일한 데이터 (matching_data)
matching_data = df[mask]

# 동일하지 않은 데이터 (disjoint_data)
disjoint_data = df[~mask]



In [29]:
matching_data

,id,paragraph,question,choices,answer,question_plus,cleaned_category
181,generation-for-nlp-662,대체출산이 달성된 지 오래된 후에도 인구가 오랫동안 계속해서 증가하는 경향은...,대체출산이 달성된 지 오래된 후에도 인구가 오랫동안 계속해서 증가하는 경향은?,"['영의 인구 증가', '고성장 속도', '항상성 고원', '인구학적 모멘텀']",4,NaN,사회-인구와사회
182,generation-for-nlp-663,거리가 멀어짐에 따라 이동이 감소하는 경향은...,거리가 멀어짐에 따라 이동이 감소하는 경향은?,"['유출 요인', '유인 요인', '거리 감쇠', '이수 선택성']",3,NaN,사회-경제
183,generation-for-nlp-664,다국적 기업은 운영을 분산하여...,다국적 기업의 운영 방식에 대한 설명으로 옳은 것은?,"['생산비용이 가장 높은 제품을 제조한다', '경제적인 곳에서 회계 및 연구 서비스...",3,NaN,사회-경제
185,generation-for-nlp-667,인도 바라나시(베나레스)와 사우디아라비아의 메카가 비슷한 도시인 이유는...,인도 바라나시(베나레스)와 사우디아라비아의 메카가 비슷한 도시인 이유는?,"['구 영국 식민지였던 국가의 수도', '수많은 순례자의 목적지', '세계 경제의 ...",2,NaN,사회-문화
191,generation-for-nlp-675,"선거인단 제도의 구조를 감안할 때, 대선 후보들은 주로 어떤 경향을 갖습니까?","선거인단 제도의 구조를 감안할 때, 대선 후보들은 주로 어떤 경향을 갖습니까?","['격전지를 피하고 “안전한” 주에 선거운동을 집중', '인구가 많은 주에서 더 많...",2,NaN,사회-정치와법
193,generation-for-nlp-677,수정헌법 제1조는 언론과 종교의 자유를 보호하는 것으로 가장 잘 알려져 있는데.,수정헌법 제1조가 보호하는 또 다른 권리는 무엇입니까?,"['모든 형사 사건에서의 적법절차', '정부가 정당하게 보상하지 않는 한 개인 재산...",4,NaN,사회-정치와법
198,generation-for-nlp-683,당좌 예금 계좌에 1000달러가 입금되고 초과 준비금이 700달러 증가하면 준비율은...,당좌 예금 계좌에 1000달러가 입금되고 초과 준비금이 700달러 증가하면 준비율은...,"['70%.', '30%.', '40%.', '90%.']",2,NaN,사회-경제
203,generation-for-nlp-688,총 공급 곡선이 왼쪽으로 이동하면 균형 물가 수준과 균형 산출량은 어떻게 되겠습니까...,총 공급 곡선이 왼쪽으로 이동하면 균형 물가 수준과 균형 산출량은 어떻게 되겠습니까?,"['균형 물가 수준은 증가하는 반면 균형 산출량은 감소한다.', '균형 물가 수준은...",1,NaN,사회-경제
206,generation-for-nlp-691,단기 총 공급 곡선이 수평인 경우 그 이유는 다음과 같습니다.,단기 총 공급 곡선이 수평인 경우 그 이유는?,['임금과 물가를 올리지 않고도 생산량을 늘릴 수 있도록 유휴 자원이 많이 존재한다...,1,NaN,사회-경제
208,generation-for-nlp-693,사람들이 가까운 시일 내에 특정 제품의 가격이 인상될 것으로 예상하는 경우,사람들이 가까운 시일 내에 특정 제품의 가격이 인상될 것으로 예상하는 경우에 대한 ...,"['이는 현재 제품 수요에는 영향을 미치지 않는다.', '이는 제품에 대한 수요를 ...",4,NaN,사회-경제


In [30]:
disjoint_data

,id,paragraph,question,choices,answer,question_plus,cleaned_category
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,NaN,사회-한국사
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,사회-동아시아사
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,사회-동아시아사
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,사회-한국지리
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,사회-동아시아사
...,...,...,...,...,...,...,...
2016,generation-for-nlp-2893,"“헐값에 팔리는 냉동 오렌지주스만 잔뜩 사가고, 쿠폰을 오려 모았으며, 구멍 난 스...",잭 맥도널드가 남긴 유산의 총액은 얼마인가?,"['1억8760만달러', '5000만달러', '2억달러', '1억달러', '3억달러']",1,NaN,사회-경제
2017,generation-for-nlp-2894,"넷기어코리아(지사장 김진겸, 이하 넷기어)가 뮤럴 캔버스를 가지고 넷기어 SNS에서...",넷기어가 모집하는 '뮤럴 공식 도슨트'의 주요 역할은 무엇인가?,"['전시물 설명 및 안내', '고객 서비스 제공', '제품 판매 촉진', '온라인 ...",4,NaN,사회-문화
2018,generation-for-nlp-2895,서울 성동구 옥수동과 금호동은 맞붙어 있는 동네지만 아파트값은 같은 면적에서 1억원...,옥수동과 금호동의 아파트값 차이가 발생한 주된 이유는 무엇인가?,"['옥수동의 재개발이 먼저 이루어졌다', '금호동의 인구가 더 많다', '옥수동이 ...",1,NaN,사회-경제
2019,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?,"['셀트리온', '삼성전자', 'LG화학', '한화그룹', '현대자동차']",1,NaN,사회-경제


In [31]:
disjoint_data.to_csv("last_data.csv", index=False)